In [87]:
import pandas as pd
import numpy as np
import re

In [88]:
# Para el manejo de este archivo se define como regla general

# error = true
# ok = false

# Siempre que las validaciones arrojen True significa que hay datos sin calidad.

In [89]:
# Ruta al archivo CSV
ruta_csv = 'data/temp/kna1.csv'

# Cargar el archivo CSV en un DataFrame con opciones adicionales
kna1 = pd.read_csv(
    ruta_csv,
    sep=',',  # Delimitador
    header=0,  # Primera fila como encabezado
    dtype={'Tratamiento': str, 'Número fiscal 5': str, 'Teléfono 1': str},  # Tipos de datos
    na_values=['NA', 'N/A']  # Valores a considerar como NaN
)

In [90]:
kna1_renamed =  kna1.rename(columns={'Tipo NIF': 'nif'})

# Ruta al archivo CSV
ruta_csv = 'data/generalTabes/nif.csv'

# Cargar el archivo CSV en un DataFrame con opciones adicionales
nif = pd.read_csv(
    ruta_csv,
    sep=',',  # Delimitador
    header=0,  # Primera fila como encabezado
    
)


nif_renamed =  nif.rename(columns={'nif': 'nif'})


# Realizar el merge
kna1_renamed = pd.merge(kna1_renamed, nif_renamed, on='nif', how='left')

# Seleccionar columnas específicas
validacionNif = ['Cliente','nif', 'descripcion']
resultado_seleccionado = kna1_renamed[validacionNif]

In [91]:
kna1_renamed =  kna1_renamed.rename(columns={'Clase de impuesto': 'claseImp'})

# Ruta al archivo CSV
ruta_csv = 'data\generalTabes\claseImpuesto.csv'

# Cargar el archivo CSV en un DataFrame con opciones adicionales
impuesto = pd.read_csv(
    ruta_csv,
    sep=',',  # Delimitador
    header=0,  # Primera fila como encabezado
)


impuesto_renamed =  impuesto.rename(columns={'clase': 'claseImp'})


# Realizar el merge
resultado = pd.merge(kna1_renamed, impuesto_renamed, on='claseImp', how='left')
resultado['tipoPerona'] = resultado['descripcion impuesto'].str[:2]

In [92]:
# tratamiento empresa no tenga clase de impuestos pn
cond1 = resultado['Tratamiento'] == 'Empresa'
cond2 = resultado['tipoPerona'] == 'PN'

resultado['tratamiento_x_claseImpto'] = np.where(cond1 & cond2, True, False)

In [93]:
# tratamiento empresa no tenga tipo nif <> 31
cond1 = resultado['Tratamiento'] == 'Empresa'
cond2 = resultado['nif'] != 31

resultado['tratamiento_x_nif'] = np.where(cond1 & cond2,True,False)

In [94]:
# tratamiento empresa no tenga medios magneticos (nombre1 y nombre2)

cond1 = resultado['Tratamiento'] == 'Empresa'
cond2 = resultado['Nombre 3'] == 'NaN'
cond3 = resultado['Nombre 4'] == 'NaN'

resultado['tratamiento_x_medMag'] = np.where(cond1 & cond2 & cond3,True,False)

In [95]:
# nif 5 no este vacio

cond1 = resultado['Número fiscal 5'].isnull()


resultado['error_nif5'] = np.where(cond1,True,False)

In [96]:
# los numeros de telefono no tengan guiones (-)

cond1 = resultado['Teléfono 1']

cond1 = cond1.fillna('').astype(str)

def contains_hyphen(text):
    return '-' in text

# Aplicar la función a cada elemento de la serie
resultado['error_telefono'] = cond1.apply(contains_hyphen)

In [97]:
# los numeros de telefono estan vacios

cond1 = resultado['Teléfono 1'].isnull()


resultado['error_telefono_vacio'] = np.where(cond1,True,False)

In [98]:
# las direcciones no tengan caracteres especiales (# - grados N`)

cond1 = resultado['Calle']

cond1 = cond1.fillna('').astype(str)

def contains_hyphen(text):
    #definir los caracteres especiales que se consideran como error en la direccion
    patron = r'[#-]'
    return bool(re.search(patron, text))

# Aplicar la función a cada elemento de la serie
resultado['error_calle'] = cond1.apply(contains_hyphen)

In [99]:
# tratamiento empresa no tenga medios magneticos (nombre1 y nombre2)

    
# Crear una columna 'error_personaFisica' basada en condiciones
resultado['error_personaFisica'] = False  # Inicializar con un valor por defecto

# Aplicar condiciones
resultado.loc[(resultado['claseImp'] == 13) & (resultado['nif'] == 13) & (resultado['Persona física'].isnull()), 'error_personaFisica'] = True

resultado.loc[(resultado['claseImp'] != 13) & (resultado['nif'] == 13) & (resultado['Persona física'] == 'X'), 'error_personaFisica'] = True

resultado.loc[(resultado['claseImp'] == 13) & (resultado['nif'] != 13) & (resultado['Persona física'] == 'X'), 'error_personaFisica'] = True


In [100]:
# Seleccionar columnas específicas
vista = ['Tratamiento', 'Cliente','claseImp', 'tipoPerona', 'nif', 'descripcion', 'tratamiento_x_claseImpto']
vista = ['Tratamiento', 'Cliente','tipoPerona','tratamiento_x_claseImpto']
vista = ['Tratamiento', 'Cliente','nif','tratamiento_x_nif']
vista = ['Tratamiento', 'Cliente','nif', 'Nombre 3', 'Nombre 4', 'tratamiento_x_medMag']
vista = ['Tratamiento', 'Cliente','nif', 'Número fiscal 5','error_nif5']
vista = ['Cliente','nif', 'Teléfono 1','error_telefono']
vista = ['Cliente','nif', 'Teléfono 1','error_telefono_vacio']
vista = ['Cliente','nif', 'Calle','error_calle']
vista = ['Cliente','nif', 'claseImp', 'Persona física', 'error_personaFisica']
resultado_seleccionado = resultado[vista]


#print(resultado_seleccionado)
print(resultado_seleccionado)

     Cliente  nif  claseImp Persona física  error_personaFisica
0   11038129   13        13            NaN                 True
1   11038197   31        14            NaN                False
2   11038219   43        13              X                 True
3   11038230   13        13              X                False
4   11038465   31        14            NaN                False
5   11038479   31        14            NaN                False
6   11038501   31        14            NaN                False
7   11038517   31        14            NaN                False
8   11038783   31        13            NaN                False
9   11038858   31         8            NaN                False
10  11039031   13        13              X                False
11  11039043   13        13              X                False
12  11039046   31        14            NaN                False
13  11039080   13        13              X                False
14  11039091   13        13             